In [1]:
import sys
sys.path.append("../../wideboost/")

In [2]:
import time
from wideboost.wrappers import wxgb
import xgboost as xgb

import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval
import os

def hyperopt(
    wide,
    n_iters,
    x_train,
    x_valid,
    x_test,
    y_train,
    y_valid,
    y_test,
    y_cols
):
    os.environ['HYPEROPT_FMIN_SEED'] = '123123'
    best_val = 1000
    num_round = 500
    valid_best = 1000
    test_best = 1000
    best_iter = 0
    best_param = {}
    best_bst = None
    
    maximize = False
    eval_metric = 'many_logloss'
    
    spc = (
        {
            'btype': hp.choice('btype',['I', 'In', 'R', 'Rn']),
            'extra_dims': hp.choice('extra_dims', range(-len(y_cols)+1,6)),
            'beta_eta': hp.loguniform('beta_eta', -7, 0),
            'objective': hp.choice('objective',['manybinary:logistic']),
            'eval_metric':hp.choice('eval_metric',[[eval_metric]]),
            'eta': hp.loguniform('eta', -7, 0),
            'max_depth' : hp.choice('max_depth',range(1,11)),
            'subsample': hp.uniform('subsample', 0.5, 1),
            'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
            'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
            'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
            'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
            'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
            'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
        } if wide else {
            'btype': hp.choice('btype',['I']),
            'extra_dims': hp.choice('extra_dims',[0]),
            'beta_eta': hp.choice('beta_eta', [0]),
            'objective': hp.choice('objective',['manybinary:logistic']),
            'eval_metric':hp.choice('eval_metric',[[eval_metric]]),
            'eta': hp.loguniform('eta', -7, 0),
            'max_depth' : hp.choice('max_depth',range(1,11)),
            'subsample': hp.uniform('subsample', 0.5, 1),
            'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
            'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
            'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
            'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
            'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
            'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
        }
    )
    
    def objective(param):
        nonlocal best_val
        nonlocal valid_best
        nonlocal test_best
        nonlocal best_param
        nonlocal num_round
        nonlocal best_iter
        nonlocal best_bst
        
        watchlist = [
            ((x_train, y_train),'train'),
            ((x_test, y_test), 'test'),
            ((x_valid, y_valid), 'valid')
        ]
        param['output_dim'] = len(y_cols)
        param['nthread'] = 2

        results = {}
        wbst = wxgb.fit(
            x_train,
            y_train,
            param, num_round, watchlist,
            evals_result=results,
            early_stopping_rounds=20,
            maximize=maximize,
            verbose_eval=200
        )
        
        new_loss = (
            min(results['valid'][eval_metric]) if not maximize
            else -max(results['valid'][eval_metric])
        )
        
        if new_loss < best_val:
            print("!!!!!!!!!NEW BEST VALUE!!!!!!")
            
            best_val = new_loss
            
            valid_best = (
                min(results['valid'][eval_metric]) if not maximize
                else max(results['valid'][eval_metric])
            )
            test_best = results['test'][eval_metric][
                np.nanargmin(results['valid'][eval_metric]) if not maximize
                else np.nanargmax(results['valid'][eval_metric])
            ]
            best_iter = wbst.xgbobject.best_iteration
            
            best_param = param.copy()
            best_bst = wbst
        
        return {'loss': new_loss, 'status': STATUS_OK }
    
    best = fmin(objective,
        space=spc,
        algo=tpe.suggest,
        max_evals=n_iters)
    
    return {
        'valid_eval': valid_best,
        'test_eval': test_best,
        'best_iter': best_iter,
        'param': best_param.copy(),
        'bst': best_bst
    }

In [3]:
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=False,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    
for i in ds_test.batch(60000):
    b = i

In [4]:
xup = a[0].numpy().reshape([-1,28*28])
yup = a[1].numpy()

xdown = b[0].numpy().reshape([-1,28*28])
ydown = b[1].numpy()

def _onehot(Y):
    b = np.zeros((Y.shape[0], Y.max().astype(int)+1))
    b[np.arange(Y.shape[0]),Y.astype(int).flatten()] = 1
    return b
def sort_fix(li):
    li.sort()
    return np.array(li)

def get_data(xtrain, ytrain, seed, sizes):
    np.random.seed(seed)
    train_idx = np.random.choice(range(xtrain.shape[0]), round(xtrain.shape[0] * 0.7), replace=False)
    remaining_idx = sort_fix(list(set(range(xtrain.shape[0])) - set(train_idx)))
    valid_idx = np.random.choice(remaining_idx, round(len(remaining_idx) * 0.5), replace=False)
    remaining_idx = sort_fix(list(set(range(xtrain.shape[0])) - set(train_idx) - set(valid_idx)))
    test_idx = np.random.choice(remaining_idx, len(remaining_idx), replace=False)

    train_idx = np.random.choice(train_idx, sizes[0], replace=False)
    valid_idx = np.random.choice(valid_idx, sizes[1], replace=False)
    test_idx = np.random.choice(test_idx, sizes[2], replace=False)

    x_train = xtrain[train_idx[:,None], features]
    x_valid = xtrain[valid_idx[:,None], features]
    x_test = xtrain[test_idx[:,None], features]           
    
    y_train = _onehot(ytrain[train_idx])
    y_valid = _onehot(ytrain[valid_idx])
    y_test =  _onehot(ytrain[test_idx])

    dtrain = xgb.DMatrix(xtrain[train_idx[:,None], features],label=ytrain[train_idx])
    dvalid = xgb.DMatrix(xtrain[valid_idx[:,None], features],label=ytrain[valid_idx])
    dtest = xgb.DMatrix(xtrain[test_idx[:,None], features],label=ytrain[test_idx])

    return dtrain, dvalid, dtest, x_train, x_valid, x_test, y_train, y_valid, y_test

In [5]:
def get_upstream(seed):
    return get_data(xup, yup, seed, sizes = [3_000, 2_500, 2_000])

def get_downstream(seed):
    return get_data(xdown, ydown, seed, sizes = [5_00, 1_500, 1_000])

In [6]:
xgb.__version__

'1.6.2'

In [7]:
%%time
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

iters = 30
upstream_results = []
downstream_results = []
for est in range(iters):
    t0 = time.time()
    np.random.seed(10000+est)
    features = np.random.choice(range(xup.shape[1]), 200, replace=False)
    downstream_dim = np.random.choice(range(9), 5, replace=False)

    dtrain, dvalid, dtest, x_train, x_valid, x_test, y_train, y_valid, y_test = get_upstream(10000 + est)
    upstream_mask = np.ones([10], dtype=bool)
    upstream_mask[downstream_dim] = False
    y_train = y_train[:, upstream_mask]
    y_valid = y_valid[:, upstream_mask]
    y_test = y_test[:, upstream_mask]
    
    o = hyperopt(
        True,
        15,
        x_train,
        x_valid,
        x_test,
        y_train,
        y_valid,
        y_test,
        y_cols=range(np.sum(upstream_mask))
    )
    upstream_results.append(o)
    
    dtrain, dvalid, dtest, x_train, x_valid, x_test, newy_train, newy_valid, newy_test = get_downstream(10000 + est)
    emb_train = o['bst'].xgbobject.predict(dtrain)
    emb_valid = o['bst'].xgbobject.predict(dvalid)
    emb_test = o['bst'].xgbobject.predict(dtest)

    downstream_mask = np.zeros([10], dtype=bool)
    downstream_mask[downstream_dim] = True
    newy_train = newy_train[:, downstream_mask]
    newy_valid = newy_valid[:, downstream_mask]
    newy_test =  newy_test[:, downstream_mask]
    
    d = hyperopt(
        False,
        15,
        emb_train.reshape([x_train.shape[0], -1]),
        emb_valid.reshape([x_valid.shape[0], -1]),
        emb_test.reshape([x_test.shape[0], -1]),
        newy_train,
        newy_valid,
        newy_test,
        y_cols=range(np.sum(downstream_mask))
    )
    t1 = time.time()
    d['time'] = t1 - t0
    downstream_results.append(d)


[0]	train-many_logloss:0.67784	test-many_logloss:0.67790	valid-many_logloss:0.67776
[200]	train-many_logloss:0.18307	test-many_logloss:0.20519	valid-many_logloss:0.20752
[400]	train-many_logloss:0.08415	test-many_logloss:0.11863	valid-many_logloss:0.12396
[499]	train-many_logloss:0.06069	test-many_logloss:0.09969	valid-many_logloss:0.10589
!!!!!!!!!NEW BEST VALUE!!!!!!                         
[0]	train-many_logloss:0.68649	test-many_logloss:0.68649	valid-many_logloss:0.68647
[200]	train-many_logloss:0.38354	test-many_logloss:0.38456	valid-many_logloss:0.38548
[400]	train-many_logloss:0.27888	test-many_logloss:0.28143	valid-many_logloss:0.28333
[499]	train-many_logloss:0.24916	test-many_logloss:0.25262	valid-many_logloss:0.25499
[0]	train-many_logloss:0.69213	test-many_logloss:0.69213	valid-many_logloss:0.69212
[200]	train-many_logloss:0.51219	test-many_logloss:0.51466	valid-many_logloss:0.51378
[400]	train-many_logloss:0.40629	test-many_logloss:0.41072	valid-many_logloss:0.40923
[499]

../../wideboost/wideboost/evals/classification.py:82: RuntimeWarning: divide by zero encountered in log
  return -np.mean(np.log(p))

../../wideboost/wideboost/evals/classification.py:82: RuntimeWarning: invalid value encountered in log
  return -np.mean(np.log(p))

../../wideboost/wideboost/evals/classification.py:80: RuntimeWarning: overflow encountered in exp
  p = 1/(1 + np.exp(-logits))

../../wideboost/wideboost/objectives/manybinarylogloss.py:14: RuntimeWarning: overflow encountered in exp
  P = 1 / (1 + np.exp(-logits))



[20]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
[0]	train-many_logloss:0.68894	test-many_logloss:0.68895	valid-many_logloss:0.68892
[200]	train-many_logloss:0.25829	test-many_logloss:0.26122	valid-many_logloss:0.26097
[400]	train-many_logloss:0.24631	test-many_logloss:0.25002	valid-many_logloss:0.25045
[499]	train-many_logloss:0.24274	test-many_logloss:0.24693	valid-many_logloss:0.24744
[0]	train-many_logloss:0.67633	test-many_logloss:0.67636	valid-many_logloss:0.67619
[62]	train-many_logloss:0.16104	test-many_logloss:0.20327	valid-many_logloss:0.21174
[0]	train-many_logloss:0.23817	test-many_logloss:0.27569	valid-many_logloss:0.27314
[21]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
[0]	train-many_logloss:0.65619	test-many_logloss:0.65628	valid-many_logloss:0.65595
[108]	train-many_logloss:0.08622	test-many_logloss:0.14754	valid-many_logloss:0.15680
[0]	train-many_logloss:0.49622	test-many_logloss:0.49641	valid-many_logloss:0.49565
[160]

[499]	train-many_logloss:0.27462	test-many_logloss:0.28111	valid-many_logloss:0.27808
[0]	train-many_logloss:0.69207	test-many_logloss:0.69207	valid-many_logloss:0.69207
[200]	train-many_logloss:0.50459	test-many_logloss:0.50536	valid-many_logloss:0.50485
[400]	train-many_logloss:0.39447	test-many_logloss:0.39598	valid-many_logloss:0.39519
[499]	train-many_logloss:0.35735	test-many_logloss:0.35927	valid-many_logloss:0.35837
[0]	train-many_logloss:0.36114	test-many_logloss:0.36010	valid-many_logloss:0.36227
[19]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
[0]	train-many_logloss:0.68904	test-many_logloss:0.68903	valid-many_logloss:0.68902
[200]	train-many_logloss:0.24266	test-many_logloss:0.24331	valid-many_logloss:0.24411
[400]	train-many_logloss:0.23389	test-many_logloss:0.23537	valid-many_logloss:0.23571
[499]	train-many_logloss:0.23022	test-many_logloss:0.23213	valid-many_logloss:0.23217
[0]	train-many_logloss:0.67611	test-many_logloss:0.67604	valid-many_loglo

[400]	train-many_logloss:0.10485	test-many_logloss:0.14176	valid-many_logloss:0.13804
[499]	train-many_logloss:0.07732	test-many_logloss:0.11945	valid-many_logloss:0.11566
!!!!!!!!!NEW BEST VALUE!!!!!!                         
[0]	train-many_logloss:0.68648	test-many_logloss:0.68648	valid-many_logloss:0.68637
[19]	train-many_logloss:inf	test-many_logloss:inf	valid-many_logloss:inf
[0]	train-many_logloss:0.69221	test-many_logloss:0.69222	valid-many_logloss:0.69220
[200]	train-many_logloss:0.50555	test-many_logloss:0.50738	valid-many_logloss:0.50415
[400]	train-many_logloss:0.39719	test-many_logloss:0.40045	valid-many_logloss:0.39498
[499]	train-many_logloss:0.36070	test-many_logloss:0.36458	valid-many_logloss:0.35825
[0]	train-many_logloss:0.37817	test-many_logloss:0.38443	valid-many_logloss:0.35652
[20]	train-many_logloss:nan	test-many_logloss:nan	valid-many_logloss:nan
[0]	train-many_logloss:0.68898	test-many_logloss:0.68900	valid-many_logloss:0.68892
[200]	train-many_logloss:0.26738	

...

[400]	train-many_logloss:0.25947	test-many_logloss:0.33044	valid-many_logloss:0.32509
[499]	train-many_logloss:0.21370	test-many_logloss:0.29680	valid-many_logloss:0.29044
[0]	train-many_logloss:0.60359	test-many_logloss:0.60381	valid-many_logloss:0.60419
[134]	train-many_logloss:0.07542	test-many_logloss:0.21759	valid-many_logloss:0.20003
[0]	train-many_logloss:0.69197	test-many_logloss:0.69208	valid-many_logloss:0.69209
[200]	train-many_logloss:0.50580	test-many_logloss:0.52641	valid-many_logloss:0.52560
[400]	train-many_logloss:0.38604	test-many_logloss:0.42368	valid-many_logloss:0.42190
[499]	train-many_logloss:0.34168	test-many_logloss:0.38707	valid-many_logloss:0.38485
[0]	train-many_logloss:0.68959	test-many_logloss:0.68959	valid-many_logloss:0.68960
[200]	train-many_logloss:0.36309	test-many_logloss:0.36869	valid-many_logloss:0.36920
[400]	train-many_logloss:0.28649	test-many_logloss:0.29973	valid-many_logloss:0.29906
[499]	train-many_logloss:0.26813	test-many_logloss:0.28526	v

[0]	train-many_logloss:0.51396	test-many_logloss:0.52564	valid-many_logloss:0.52465
[42]	train-many_logloss:0.01645	test-many_logloss:0.19957	valid-many_logloss:0.18646
[0]	train-many_logloss:0.69087	test-many_logloss:0.69115	valid-many_logloss:0.69113
[200]	train-many_logloss:0.39084	test-many_logloss:0.42880	valid-many_logloss:0.42827
[400]	train-many_logloss:0.24380	test-many_logloss:0.30651	valid-many_logloss:0.30517
[499]	train-many_logloss:0.19741	test-many_logloss:0.27002	valid-many_logloss:0.26831
[0]	train-many_logloss:0.59645	test-many_logloss:0.59941	valid-many_logloss:0.59892
[168]	train-many_logloss:0.04561	test-many_logloss:0.17364	valid-many_logloss:0.16117
!!!!!!!!!NEW BEST VALUE!!!!!!                                          
[0]	train-many_logloss:0.69191	test-many_logloss:0.69203	valid-many_logloss:0.69202
[200]	train-many_logloss:0.49770	test-many_logloss:0.51810	valid-many_logloss:0.51732
[400]	train-many_logloss:0.37330	test-many_logloss:0.40941	valid-many_logloss

!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.20659	test-many_logloss:0.36197	valid-many_logloss:0.36203
[20]	train-many_logloss:0.00010	test-many_logloss:nan	valid-many_logloss:nan
[0]	train-many_logloss:0.67384	test-many_logloss:0.67539	valid-many_logloss:0.67518
[196]	train-many_logloss:0.04765	test-many_logloss:0.18550	valid-many_logloss:0.18301
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.52385	test-many_logloss:0.53391	valid-many_logloss:0.53266
[35]	train-many_logloss:0.03521	test-many_logloss:0.19829	valid-many_logloss:0.20399
[0]	train-many_logloss:0.69093	test-many_logloss:0.69118	valid-many_logloss:0.69115
[200]	train-many_logloss:0.39816	test-many_logloss:0.43326	valid-many_logloss:0.43330
[400]	train-many_logloss:0.25382	test-many_logloss:0.31393	valid-many_logloss:0.31394
[499]	train-many_logloss:0.20794	test-many_logloss:0.27841	valid-many_logloss:0.27833
[0]	train-many_l

[499]	train-many_logloss:0.26884	test-many_logloss:0.28770	valid-many_logloss:0.28316
[0]	train-many_logloss:0.68824	test-many_logloss:0.68864	valid-many_logloss:0.68858
[200]	train-many_logloss:0.24161	test-many_logloss:0.30016	valid-many_logloss:0.30012
[400]	train-many_logloss:0.12504	test-many_logloss:0.22354	valid-many_logloss:0.22481
[499]	train-many_logloss:0.09772	test-many_logloss:0.21127	valid-many_logloss:0.21353
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.20657	test-many_logloss:0.36387	valid-many_logloss:0.37509
[21]	train-many_logloss:0.00009	test-many_logloss:nan	valid-many_logloss:nan
[0]	train-many_logloss:0.67358	test-many_logloss:0.67571	valid-many_logloss:0.67554
[196]	train-many_logloss:0.04940	test-many_logloss:0.20253	valid-many_logloss:0.20871
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.52924	test-many_logloss:0.53908	valid-many_logloss:0.53795
[37]	train-many

[0]	train-many_logloss:0.68978	test-many_logloss:0.68985	valid-many_logloss:0.68982
[200]	train-many_logloss:0.36290	test-many_logloss:0.37261	valid-many_logloss:0.37142
[400]	train-many_logloss:0.27389	test-many_logloss:0.29003	valid-many_logloss:0.29008
[499]	train-many_logloss:0.25156	test-many_logloss:0.27000	valid-many_logloss:0.27122
[0]	train-many_logloss:0.68796	test-many_logloss:0.68827	valid-many_logloss:0.68832
[200]	train-many_logloss:0.21920	test-many_logloss:0.26661	valid-many_logloss:0.27079
[400]	train-many_logloss:0.09987	test-many_logloss:0.17401	valid-many_logloss:0.18172
[499]	train-many_logloss:0.07354	test-many_logloss:0.15837	valid-many_logloss:0.16670
!!!!!!!!!NEW BEST VALUE!!!!!!                                         
[0]	train-many_logloss:0.20374	test-many_logloss:0.32488	valid-many_logloss:0.34060
[22]	train-many_logloss:0.00007	test-many_logloss:0.31925	valid-many_logloss:0.34180
[0]	train-many_logloss:0.67283	test-many_logloss:0.67492	valid-many_logloss:

In [10]:
import pandas as pd
pd.DataFrame(downstream_results)

,valid_eval,test_eval,best_iter,param,bst,time
0,0.154639,0.160961,191,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,363.593484
1,0.170552,0.175058,186,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,393.794683
2,0.166721,0.153719,206,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,329.097808
3,0.211430,0.204772,164,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,336.249547
4,0.157904,0.161243,201,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,326.134475
5,0.206570,0.209191,158,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,296.688165
6,0.158853,0.142701,173,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,318.088953
7,0.225447,0.211552,151,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,351.162375
8,0.151542,0.167243,199,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,331.457166
9,0.193611,0.210262,133,"{'alpha': 0, 'beta_eta': 0, 'btype': 'I', 'col...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...,325.137519


In [11]:
import pandas as pd
pd.DataFrame(upstream_results)

,valid_eval,test_eval,best_iter,param,bst
0,0.097179,0.083482,101,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
1,0.070332,0.076774,94,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
2,0.086163,0.097289,129,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
3,0.061280,0.065674,102,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
4,0.072043,0.081523,84,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
5,0.054762,0.058478,84,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
6,0.093868,0.095969,114,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
7,0.056510,0.059359,94,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
8,0.078565,0.072787,93,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
9,0.082184,0.075347,84,"{'alpha': 0, 'beta_eta': 0.017592237441123558,...",<wideboost.wrappers.wxgb.wxgb object at 0x7fa3...
